# Глубокое обучение на TensorFlow

Вы написали много кода во второй части задания, чтобы реализовать весь набор функций нейронной сети. Dropout, блочная нормализация и двумерные свертки являются рабочими лошадками глубокого обучения в компьютерном зрении. Вы также приложили много усилий, чтобы сделать ваш код эффективным и векторизованным.

Однако в последней части этого задания мы отложим ваш прекрасный код и вместо этого перейдем к одной из двух популярных платформ глубокого обучения: в этом случае TensorFlow (или PyTorch, если вы перейдете соответсвующему блокноту)


#### Что такое TensorFlow?
TensorFlow - это система для вычислений над тензорными объектами с использованием вычислительных графов  и поддержкой выполнения обратного распространения. Тензоры представляют собой n-мерные массивы, аналогичные numpy ndarray.

#### Зачем изучать  TensorFlow?
* Наш код теперь сможет исполняться на графических процессорах! В этом случае обучение будет проходить гораздо быстрее. 
* Мы хотим, чтобы Вы были готовы использовать один из развитых фреймворков для своих проектов, чтобы Вы могли проводить эксперименты эффективнее, чем если бы Вы писали каждую функцию вручную.
* Мы хотим, чтобы Вы стояли на плечах гигантов! TensorFlow и PyTorch - отличные фреймворки, которые сделают вашу жизнь намного проще.
* Мы хотим, чтобы Вы  ознакомились с подходом к кодированию глубокого обучения, который применяется  в академических кругах или в промышленности.


## Как изучать TensorFlow?

Есть много прекрасных руководств по изучению  TensorFlow, включая руководство Google (https://www.tensorflow.org/get_started/get_started).

Этот блокнот также послужит Вам руководством, что вам нужно сделать, чтобы обучать модели в TensorFlow. Просмотрите в  конце этого блокнота некоторые ссылки на полезные руководства, если вы хотите узнать больше или если Вам требуются дополнительные разъяснения.


# Содержание

Этот блокнот состоит из 5 частей. Мы будем рассматривать  TensorFlow на трех разных уровнях абстракции, что должно способствовать лучшему пониманию.

1. Подготовка: загрузка множества данных CIFAR-10.
2. Базовый интерфейс (barebone) TensorFlow: непосредственный интерфейс (barebone) вычислительных графов TensorFlow.
3. Модель интерфейса Keras: модель интерфейса  `tf.keras.Model` для определения произвольной архитектуры нейронной сети.
4. Последовательный интерфейс Keras Sequential: удобный интерфейс `tf.keras.Sequential` для определения линейной структуры сети прямого распространения.
5. Решение задач с использованием CIFAR-10: построение собственной сети, чтобы получить максимально возможную точность классификации для базы изображений CIFAR-10. Вы можете экспериментировать с любым слоем, оптимизатором, гиперпараметрами или другими расширенными функциями.

Сравнительная таблица:

| Интерфейс     | Гибкость    | Простота    |
|---------------|-------------|-------------|
| barebone| Высокая     | Низкая      |
| `tf.keras.Model`     | Высокая| Средняя   |
| `tf.keras.Sequential`| Низкая | Высокая   |

# Часть I: Подготовительная

Сначала загрузим набор данных CIFAR-10. Это может занять несколько минут для загрузки при первом запуске, но после этого файлы должны быть кэшированы на диске, а загрузка должна быть быстрее.

В предыдущих частях задания мы использовали специфический код для загрузки и чтения набора данных CIFAR-10; однако пакет `tf.keras.datasets` в TensorFlow предоставляет предустановленные  утилиты для загрузки многих распространенных наборов данных.

Для целей  задания мы по-прежнему будем писать собственный код для предварительной обработки данных и итерации на данных по мини-блокам. Пакет `tf.data` в TensorFlow предоставляет инструменты для автоматизации этого процесса, но работа с этим пакетом создает некоторые сложности и выходит за рамки этого задания. Однако использование `tf.data` может быть намного эффективнее, чем простой подход, используемый в этом блокноте, поэтому вы должны использовать его в своих проектах.

In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Извлекает набор данных CIFAR-10 из Интернета и выполняет предварительную
    обработку данных для двухслойного нейроклассификатора. 
    Это те же шаги, что мы использовали для SVM и которые собраны в одной функции.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


### Подготовка: объект Dataset

Для нашего удобства мы определим упрощенный класс `Dataset`, который позволит нам перебирать данные и метки. Это не самый гибкий или эффективный способ перебора данных, но он будет служить нашим целям.

In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Конструирование объект Dataset для итераций по набору данных X и по меткам y
        
         Входы:
         - X: numpy массив данных любой формы
         - y: numpy массив меток любой формы, но с y.shape [0] == X.shape [0]
         - batch_size: целое число, указывающее количество элементов миниблока
         - shuffle: (необязательно) Логическое значение, следует ли перетасовывать данные
        на каждой эпохе
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# Мы можем выполнять интерации так:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


Вы можете опционально использовать GPU, установив флаг **USE_GPU в True** ниже. Для этого задания не обязательно использовать GPU; если вы работаете в Google Cloud, то мы рекомендуем вам не использовать графический процессор, так как аренда будет значительно дороже.

In [5]:
# Set up some global variables
USE_GPU = False

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /cpu:0


# Часть II: Базовый интерфейс (вarebone) TensorFlow

TensorFlow поставляется с различными API-интерфейсами, которые делают его очень удобным для определения и обучения нейронных сетей; мы рассмотрим некоторые из этих конструкций в Части III и Части IV этого блокнота. В этом разделе рассмотрим создание модели с базовыми конструкциями TensorFlow, чтобы помочь вам лучше понять, что происходит под капотом базового API-интерфейса.

TensorFlow, в первую очередь, - это фреймворк для  работы со **статическими вычислительными графами**. Ребрами  вычислительного графа являются тензоры, которые хранят n-мерные массивы; узлы  графа представляют собой функции, которые применяются к тензорам, когда выполняются вычисления в соответствии с вычислительным графом.

Это означает, что типичная программа TensorFlow содержит в две разные фазы:

1. Создание вычислительного графа, который описывает вычисления, подлежащие выполнению. Этот этап фактически не выполняет никаких вычислений; он просто создает символическое представление ваших вычислений. Этот этап обычно определяет один или несколько объектов типа «placeholder», которые представляют входные данные вычислительного графа.
2. Многократное исполнение вычислительного графа. Каждый раз, когда граф исполняется, вы указываете, какие части графа вы хотите вычислить, и передаёте словарь `feed_dict`, который поставляет конкретные значения любому объекту ` placeholder `на графе.



### Разминка с TensorFlow: функция flatten

Определим простую функцию «уплощения» `flatten`, которая реформатирует данные изображений для использования в полносвязанной сети.

В TensorFlow данные для сверточных карт признаков  обычно хранятся в тензоре формы N x H x W x C, где:

- N - количество точек данных (размер мини-блока)
- H - высота карты 
- W - ширина карты 
- C - количество каналов карты

Это правильный способ представления данных для двумерной свертки, которая учитывает пространственные отношения между признаками изображений. Однако, когда мы используем полносвязанные слои нейронов для обработки изображений,то требуется, чтобы каждое  изображение представлялось вектором.Поэтому необходимо реформатировать изображение размером «H x W x C» в один длинный вектор. Функция реформатирования (flatten) ниже сначала определяет значение N заданного блока данных, а затем возвращает реформатированное представление этих данных. Это представление формируется аналогично методу «reshape» numpy: изменяется  размер x на N x ??, где ?? некоторое значение (в рассматриваемом случае это H x W x C, но нам не требуется указывать его явно).

**ПРИМЕЧАНИЕ**: TensorFlow и PyTorch различаются своими представлениями тензоров по умолчанию; TensorFlow использует представление N x H x W x C, а PyTorch использует N x C x H x W.

In [6]:
def flatten(x):
    """    
    Входные данные:
     - Тензор формы (N, D1, ..., DM)
    
     Выход:
     Тензор формы (N, D1 * ... * DM)
    """
    N = tf.shape(x)[0]
    return tf.reshape(x, (N, -1))

In [7]:
def test_flatten():
    # Очистка текущего графа TensorFlow.
    tf.reset_default_graph()
    
   
    # Этап I: Определение графа TensorFlow, описывающего наши вычисления.
    # В этом случае вычисление тривиально: мы просто хотим реформатировать
    # Тензор, используя функцию flatten, определенную выше.
   
    # У нас будет один вход x. Нам еще неизвестно его
    # значение, поэтому мы определяем объект placeholder (местозаполнитель),
    # который будет хранить это значение,  когда граф будет исполняться. 
    # Затем этот "заполнитель" тензора передается функции flatten, которая вернет 
    # новый Тензор, предназначенный для  хранения плоского представления x.
    # Вычисления на графе пока не выполняются.   
    # Менеджер контекста tf.device сообщает TensorFlow, следует ли размещать эти тензоры
    # на CPU или графическом процессоре GPU.
    
    
    with tf.device(device):
        x = tf.placeholder(tf.float32)
        x_flat = flatten(x)
    
    # На данный момент мы просто построили граф, описывающий наши вычисления,
    # но мы пока ничего не вычислили. Если мы напечатаем x и x_flat
    # мы увидим, что они не содержат никаких данных; они являются тензорами TensorFlow
    # представляющими значения, которые будут вычисляться в ходе исполнения графа.

    
    print('x: ', type(x), x)
    print('x_flat: ', type(x_flat), x_flat)
    print()
    
   
    # Для фактического исполнения графа необходимо использовать объект Session TensorFlow 
    with tf.Session() as sess:
        # Создаем конкретные значения входных данных x с помощью numpy
        x_np = np.arange(24).reshape((2, 3, 4))
        print('x_np:\n', x_np, '\n')
    
        # Исполняем вычислительный граф для вычисления конкретного выходного значения.
        # Первый аргумент метода sess.run говорит TensorFlow, что 
        # мы хотим, чтобы он вычислил значение x_flat; feed_dict указывает
        # значения подключаемые ко всем узлам-заполнителям нашего графа.
        # Итоговое значение x_flat возвращается из sess.run как
        # numpy array.
        x_flat_np = sess.run(x_flat, feed_dict={x: x_np})
        print('x_flat_np:\n', x_flat_np, '\n')

        # Мы можем повторно использовать один и тот же граф для выполнения одних
        # и тех же вычислений с различными входными данными
        x_np = np.arange(12).reshape((2, 3, 2))
        print('x_np:\n', x_np, '\n')
        x_flat_np = sess.run(x_flat, feed_dict={x: x_np})
        print('x_flat_np:\n', x_flat_np)
test_flatten()

x:  <class 'tensorflow.python.framework.ops.Tensor'> Tensor("Placeholder:0", dtype=float32, device=/device:CPU:0)
x_flat:  <class 'tensorflow.python.framework.ops.Tensor'> Tensor("Reshape:0", shape=(?, ?), dtype=float32, device=/device:CPU:0)

x_np:
 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]] 

x_flat_np:
 [[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23.]] 

x_np:
 [[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]] 

x_flat_np:
 [[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]]


### Базовый  TensorFlow: Двухслойная сеть
Теперь мы реализуем нашу первую нейронную сеть на TensorFlow: двухслойную сеть с двумя полносвязными скрытыми слоями без смещений и с ReLU нелинейностью. Пока будем использовать только низкоуровневые операторы TensorFlow для определения сети; позже мы увидим, как использовать абстракции более высокого уровня, предоставляемые `tf.keras`, чтобы упростить процесс.

Определим функцию прямого распространения  `two_layer_fc`; она будет принимать тензоры входов и весов сети и возвращать тензор оценки предсказания класса. Важно помнить, что вызов функции `two_layer_fc` **не выполняет** никаких вычислений; вместо этого она просто определяет вычислительный граф  прямого пути. Чтобы фактически запустить сеть, нам нужно открыть сеанс Session TensorFlow и передать данные на  вход вычислительного графа.

После определения  функции `two_layer_fc` мы проверим её реализацию, исполнив вычислительный граф, подав нули на входы сети и проверив форматы вывода.

Важно, чтобы вы прочитали и поняли эту реализацию.

In [8]:
def two_layer_fc(x, params):
    """
    Полносвязная нейронная сеть; архитектура:
    полносвязный слой -> ReLU ->  полносвязный слой .
    Обратите внимание, что нам сейчас нужно только определить прямое распространение; 
    TensorFlow позаботится сам о вычислении градиентов для нас.
    
    Вход сети - мини-блок данных размерности (формы):
    (N, d1, ..., dM), где d1 * ... * dM = D. 
    Скрытый слой содержит H нейронов.
    Выходной слой вычисляет оценки рейтигов для C классов.

    Входы:
     - x: тензор формы (N, d1, ..., dM),представляющий мини-блок
       входных данных.
     - params: список [w1, w2] тензоров, представляющих веса сети,
       где w1 имеет форму (D, H), а w2 имеет форму (H, C).
    
     Возвращает:
     - scores: тензор формы (N, C), представляющий оценки рейтингов 
       принадлежности классам входных данных x.  
    """
    w1, w2 = params  # распаковка параметров
    x = flatten(x)   # реформатируем x к форме (N, D)
    h = tf.nn.relu(tf.matmul(x, w1)) # Скрытый слой : форма h - (N, H)
    scores = tf.matmul(h, w2)        # Вычисление рейтингов, форма scores - (N, C)
    return scores

In [9]:
def two_layer_fc_test():
    # Вычислительный граф TensorFlow, по сути, является скрытой глобальной
    # переменной. Чтобы избежать добавления к этой переменной графа по умолчанию при 
    # повторном запуске этой ячейки Jupyter notebook,
    # очистим граф по умолчанию, прежде чем строить наш граф.
    tf.reset_default_graph()
    hidden_layer_size = 42

    # Разместим наш код вычислительного графа в контексте менежджера tf.device,
    # что позволит указывать TensorFlow, где должны размещаться тензоры.
    with tf.device(device):
        # Определяем  placehoder для входного тензора и тензоры для сетевых весов.
        # Здесь мы объявляем w1 и w2, используя tf.zeros вместо tf.placeholder, 
        # Это означает, что значения w1 и w2 будут сохранены в самом вычислительном
        # графе и тем самым будут сохраняться между прогонами графа; в
        # частности, это означает, что нам не нужно передавать значения w1 и w2
        # с использованием feed_dict, когда мы в конечном счете исполняем граф.
        x = tf.placeholder(tf.float32)
        w1 = tf.zeros((32 * 32 * 3, hidden_layer_size))
        w2 = tf.zeros((hidden_layer_size, 10))
        
        # Вызываем функцию two_layer_fc, чтобы создать вычислительный
        # граф для прямого прохода по сети.
        scores = two_layer_fc(x, [w1, w2])
    
    # Используем  numpy для подготовки данных, которые ниже будут переданы
    # вычислительный граф, путем  связи с x placeholder.
    x_np = np.zeros((64, 32, 32, 3))
    with tf.Session() as sess:
        # Вызовы tf.zeros выше фактически не создают значения
        # для w1 и w2;строка ниже заставляет TensorFlow создать экземпляры
        # значений всех тензоров (например, w1 и w2), которые хранятся в графе.
        sess.run(tf.global_variables_initializer())
        
        # Здесь мы фактически выполняем граф, используя feed_dict, чтобы передать
        # значение x_np и  привязать его  к заполнителю x;  TensorFlow вычисляет
        # значение тензора scores, которое возвращается в виде numpy массива.
        scores_np = sess.run(scores, feed_dict={x: x_np})
        print(scores_np.shape)

two_layer_fc_test()

(64, 10)


### Базовый TensorFlow: Трехслойная  ConvNet

Реализуйте функцию `three_layer_convnet`, которая будет выполнять прямое распространение для трехслойной сверточной сети. Сеть  должна иметь следующую архитектуру:

1. Сверточный слой (со смещением) с фильтрами `channel_1`, каждый размером ` KW1 x KH1` и дополнением двумя нулями, P=2
2. Нелинейность ReLU
3. Сверточный слой (со смещением) с фильтрами `channel_2`, каждый размером ` KW2 x KH2` и  дополнением одним нулем, P=1
4. Нелинейность ReLU
5. Полносвязанный слой со смещением, вычисляющий оценки рейтингов для `C` классов .


**СОВЕТ**: Для сверток: https://www.tensorflow.org/api_docs/python/tf/nn/conv2d; будьте внимательны с добавлением нулей!

**СОВЕТ**: ДЛя смещений: https://www.tensorflow.org/performance/xla/broadcasting

In [10]:
def three_layer_convnet(x, params):
    """
    Трехслойная сверточная сеть с описанной выше архитектурой.
    
    Входы:
    - x: тензор формы (N, H, W, 3), представляющий мини-блок изображений
    - params: список тензоров, представляющих веса и смещения 
      сети; должен содержать следующее:
      - conv_w1: тензор формы (KH1, KW1, 3, channel_1) -
        веса первого сверточного слоя.
      - conv_b1: тензор формы (channel_1,) - смещения
        первого сверточного слоя.
      - conv_w2: тензор формы (KH2, KW2, channel_1, channel_2)
        весовые коэффициенты второго сверточного слоя
      - conv_b2: тензор  формы (channel_2,) - смещения
        второго сверточного слоя.
      - fc_w: тензор представляющий весовые коэффициенты полносвязанного слоя.
        Укажите сами, какова должна быть форма?
      - fc_b: тензор смещений полносвязанного слоя.
        Укажите сами, какова должна быть форма?    
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ############################################################################
    # ЗАДАНИЕ: Реализуйте прямое распространение для 3-х слойной ConvNet.      #
    ############################################################################
    x_padded = tf.pad(x,[[0,0],[2,2],[2,2],[0,0]],'CONSTANT')
    conv1 = tf.nn.conv2d(x_padded,conv_w1,[1,1,1,1],padding='VALID')+conv_b1
    relu1 = tf.nn.relu(conv1)
    x_padded_1 = tf.pad(relu1,[[0,0],[1,1],[1,1],[0,0]],'CONSTANT')
    conv2 = tf.nn.conv2d(x_padded_1,conv_w2,[1,1,1,1],padding='VALID')+conv_b2
    relu2 = tf.nn.relu(conv2)
    fc_x = flatten(relu2)
    h = tf.matmul(fc_x, fc_w) + fc_b
    scores = h
    ############################################################################
    #                              КОНЕЦ ВАШЕГО КОДА                           #
    ############################################################################
    return scores

После определения  трехслойной ConvNet выше, запустите следующую ячейку, чтобы проверить вашу реализацию. Подобно двухслойной сети, используйте функцию `three_layer_convnet` для создания вычислительного графа, а затем исполните граф на блоке из нулей только для того, чтобы убедиться, что функция не дает сбоя, и создает выходы правильной формы (размерности).

Когда вы запустите эту функцию, `scores_np` должен будет иметь форму` (64, 10) `.

In [11]:
def three_layer_convnet_test():
    tf.reset_default_graph()

    with tf.device(device):
        x = tf.placeholder(tf.float32)
        conv_w1 = tf.zeros((5, 5, 3, 6))
        conv_b1 = tf.zeros((6,))
        conv_w2 = tf.zeros((3, 3, 6, 9))
        conv_b2 = tf.zeros((9,))
        fc_w = tf.zeros((32 * 32 * 9, 10))
        fc_b = tf.zeros((10,))
        params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
        scores = three_layer_convnet(x, params)

    # Входы  сверточных слоев представляют собой 4-мерные массивы с формой
    # [размер_блока, высота, ширина, каналы]
    x_np = np.zeros((64, 32, 32, 3))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        scores_np = sess.run(scores, feed_dict={x: x_np})
        print('scores_np has shape: ', scores_np.shape)

with tf.device('/cpu:0'):
    three_layer_convnet_test()

scores_np has shape:  (64, 10)


### Базовый TensorFlow: Этап обучения
Теперь определим функцию `training_step`, которая создает часть вычислительного графика,  выполняющую  шаг обучения. Это потребует трех основных действий:

1. Вычислить функцию потери
2. Вычислить градиент функции потерь по всем весам
3. Выполнить шаг обновления веса, используя (стохастический) градиентный спуск.

Обратите внимание, что шаг обновления весов сам по себе является операцией в вычислительном графе - вызовы `tf.assign_sub` в` training_step` возвращают операции TensorFlow, которые изменяют веса при их выполнении. Здесь есть тонкость - когда мы называем `sess.run`, TensorFlow не выполняет все операции в вычислительном графе; он выполняет только минимальное подмножество графа, необходимое для вычисления выходов, которые мы запрашиваем. В результате наивное вычисление потерь не приводит к выполнению операций по обновлению веса, поскольку операции, необходимые для вычисления потерь, не зависят от результата обновления весов. Чтобы решить эту проблему, мы вставляем **зависимости управления** `tf.control_dependencies` в граф, добавляя дублирующий узел «loss» к графу, который зависит от выходов операций обновления веса; это объект, который мы фактически возвращаем из функции `training_step`. В результате, попросив TensorFlow оценить значение `loss`, возвращаемое ` training_step`, также будут неявно обновляться веса сети на мини-блоке данных.


Нам нужно использовать несколько новых функций TensorFlow, чтобы сделать все это:
- Для вычисления потерь в виде кросс-энтропии мы будем использовать
`tf.nn.sparse_softmax_cross_entropy_with_logits`: https://www.tensorflow.org/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits
- Для усреднения потерь на миниблоке данных мы будем использовать `tf.reduce_mean`:
https://www.tensorflow.org/api_docs/python/tf/reduce_mean
- Для вычисления градиентов потерь по отношению к весам мы будем использовать `tf.gradients`:https://www.tensorflow.org/api_docs/python/tf/gradients
- для изменения значений весов, хранящихся в тензоре будем использовать`tf.assign_sub`:
https://www.tensorflow.org/api_docs/python/tf/assign_sub
- добавим зависимости управления к графу с помощью `tf.control_dependencies`:
https://www.tensorflow.org/api_docs/python/tf/control_dependencies

In [12]:
def training_step(scores, y, params, learning_rate):
    """
    Определяет часть вычислительного графа для шага обучения.

     Входы:
     - scores: тензор формы (N, C), содержащий классификационные рейтинги.
     - y: тензор формы (N,), содержащий корректные метки классов;
       y [i] == c означает, что c является правильным классом для scores[i].
     - params: cписок тензоров,представляющих весовые коэффициенты модели
     - learning_rate: скаляр , представляющий скорость обучения SGD
      
      
     Возвращает:
     - loss:  тензор формы () (скаляр)- потери на мини-блоке данных;
       оценка потерь также выполняет шаг алгоритма SGD (см. выше).  
    """
    # Сначала вычислим потери; первая строка дает потери для каждого примера в
    # мини-блоке, вторая усредняет потери 
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
    loss = tf.reduce_mean(losses)

    # Вычисляем градиент потерь по каждому параметру
    # сети. Это некий магический вызов функции: TensorFlow внутренне
    # обходит вычислительный граф, начиная с потерь в обратном направлении к каждому 
    # параметру и использует backpropagation, чтобы выяснить, как вычислять градиенты;
    # Затем он добавляет новые операции в вычислительный граф, которые вычисляют
    # запрошенные градиенты и возвращает список тензоров, которые будут
    # содержать запрошенные градиенты при выполнении.
    grad_params = tf.gradients(loss, params)
    
    
    # Реализуем шаг градиентного спуска по всем параметрам модели.
    new_weights = []   
    for w, grad_w in zip(params, grad_params):
        new_w = tf.assign_sub(w, learning_rate * grad_w)
        new_weights.append(new_w)

    # Вставляем управляющую зависимость, чтобы оценка потерь приводила
    # к обновлению веса; см. обсуждение выше.
    
    with tf.control_dependencies(new_weights):
        return tf.identity(loss)    

### Базовый TensorFlow: цикл обучения
Теперь мы реализуем цикл обучения, используя операции низкого уровня TensorFlow. Будем обучать модель, используя простой стохастический градиентный спуск. Функция «training_step» определила часть вычислительного графа, который связан с этапом обучения, а функция «train_part2» выполняет итерации по обучающим данным, осуществляя  шаги обучения на каждом мини-блоке и периодически оценивает точность на валидационном множестве.


In [13]:
def train_part2(model_fn, init_fn, learning_rate):
    """
    
    Входы:
     - model_fn: функция Python, которая выполняет прямое распространение,
       используя TensorFlow; она должна иметь следующую сигнатуру:
       scores = model_fn (x, params), где x - тензор, представляющий
       мини-блок данныхс изображениями, params - список тензоров, хранящих
       веса модели, scores - тензор  формы (N, C), содержащий
       рейтинги  всех элементов x.
     - init_fn: функция Python, которая инициализирует параметры модели.
       Она должна иметь сигнатуру params = init_fn (), где params - это список
       тензоров, хранящих (случайно инициализированные) веса
       модели.
     - learning_rate: вещественное значение Python, представлящее скорость обучения  SGD.
      
    """
    # Очистка графа по умолчанию
    tf.reset_default_graph()
    is_training = tf.placeholder(tf.bool, name='is_training')
    # Формирование вычислительного графа для прямого и обратного распространения,
    # и обновления весов.
    with tf.device(device):
        # Создание плейсхолдеров для данных и меток
        x = tf.placeholder(tf.float32, [None, 32, 32, 3])
        y = tf.placeholder(tf.int32, [None])
        params = init_fn()           # Инициализация параметров модели
        scores = model_fn(x, params) # Прямое распространение
        loss = training_step(scores, y, params, learning_rate)

    # Дествительное многократное исполнение графа на обучающих данных 
    with tf.Session() as sess:
        # Инициализация переменных, которые размещаются в самом графе
        sess.run(tf.global_variables_initializer())
        for t, (x_np, y_np) in enumerate(train_dset):
            # Исполнение графа на блоке обучающих данных; вызов заставляет
            # TensorFlow оценивать потери loss, что приводит к успешному шагу SGD
            feed_dict = {x: x_np, y: y_np}
            loss_np = sess.run(loss, feed_dict=feed_dict)
            
            
            # Периодически выводим потери и проверяем точность на валидационном множестве
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss_np))
                check_accuracy(sess, val_dset, x, scores, is_training)

### Базовый TensorFlow: проверка точности 
При обучении модели мы будем использовать следующую функцию для проверки точности нашей модели на обучающем и валидационном множестве данных. Обратите внимание, что эта функция принимает объект Session как один из  аргументов; это необходимо, так как функция должна фактически запускать  исполнение вычислительного графа много раз на данных, которые она загружает из набора данных `dset`.

Также обратите внимание, что мы повторно используем один и тот же вычислительный граф как для обучения, так и для оценивания модели; однако, поскольку функция `check_accuracy` никогда не вычисляет значение` loss` в вычислительном графе, часть графа, которая обновляет веса, не выполняется на валидационном множестве.





In [14]:
def check_accuracy(sess, dset, x, scores, is_training=None):
    """
    Проверяет точность классификации.
    
     Входы:
     - sess: сеанс, который будет использоваться для запуска графа
     - dset: объект Dataset, используемый для проверки точности
     - x: тензор плейсхолдер, представляющий входные изображения
     - scores: тензор, представляющий рейтинги классов на выходе модели;
     это тензор, который мы просим TensorFlow оценить.
      
     Возвращает: ничего не возвращает, но печатает точность модели
    """
    num_correct, num_samples = 0, 0
    for x_batch, y_batch in dset:
        feed_dict = {x: x_batch, is_training: 0}
        scores_np = sess.run(scores, feed_dict=feed_dict)
        y_pred = scores_np.argmax(axis=1)
        num_samples += x_batch.shape[0]
        num_correct += (y_pred == y_batch).sum()
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### Базовый TensorFlow: инициализация
Мы будем использовать следующую утилиту для инициализации матриц весов моделей, использующую метод
нормировки Кайминга (Хе).

[1] He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification
*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [15]:
def kaiming_normal(shape):
    if len(shape) == 2:
        fan_in, fan_out = shape[0], shape[1]
    elif len(shape) == 4:
        fan_in, fan_out = np.prod(shape[:3]), shape[3]
    return tf.random_normal(shape) * np.sqrt(2.0 / fan_in)

### Базовый  TensorFlow: обучение 2-х слойной нейросети
Наконец, мы готовы использовать все части, определенные выше, для обучения двухслойной полносвязной сети на множестве данных CIFAR-10.

Нам просто нужно определить функцию для инициализации весов модели и вызвать `train_part2`.

Определение весов сети представляет собой еще один важный компонент TensorFlow API: `tf.Variable`. TensorFlow Variable - это тензор-переменная, значение которой хранится в графе и сохраняется на разных  циклах исполнения вычислительного графа; однако в отличие от констант, определенных с помощью `tf.zeros` или` tf.random_normal`, значения переменной могут быть изменены при выполнении графа; эти изменения будут сохраняться в графе. Обучаемые параметры сети обычно хранятся в переменных.

Вам не нужно настраивать гиперпараметры, но вы должны достичь точности выше 40% после одной эпохи обучения.


In [16]:
def two_layer_fc_init():
    """
    Инициализирует веса двухслойной сети для использования с
    two_layer_network, определенной выше.
    
     Входы: отсутствуют
    
     Возвращает: список:
     - w1: переменная TensorFlow, представляющая веса первого слоя
     - w2: переменная TensorFlow, представляющая веса второго слоя
    """
    hidden_layer_size = 4000
    w1 = tf.Variable(kaiming_normal((3 * 32 * 32, 4000)))
    w2 = tf.Variable(kaiming_normal((4000, 10)))
    return [w1, w2]

learning_rate = 1e-2
train_part2(two_layer_fc, two_layer_fc_init, learning_rate)

Iteration 0, loss = 3.2846
Got 131 / 1000 correct (13.10%)
Iteration 100, loss = 1.9373
Got 373 / 1000 correct (37.30%)
Iteration 200, loss = 1.5112
Got 378 / 1000 correct (37.80%)
Iteration 300, loss = 1.8303
Got 360 / 1000 correct (36.00%)
Iteration 400, loss = 1.8335
Got 415 / 1000 correct (41.50%)
Iteration 500, loss = 1.8088
Got 425 / 1000 correct (42.50%)
Iteration 600, loss = 1.7331
Got 424 / 1000 correct (42.40%)
Iteration 700, loss = 1.9773
Got 448 / 1000 correct (44.80%)


### Базовый  TensorFlow: Обучение 3-х слойной  ConvNet

Теперь мы будем использовать TensorFlow для обучения трехслойной ConvNet на CIFAR-10.

Вам нужно реализовать функцию `three_layer_convnet_init`. Напомним архитектуру сети:

1. Сверточный слой (со смещением) с 32 фильтрами 5 × 5 с дополнением нулями Р=2
2. ReLU
3. Сверточный слой (со смещением) с 16 фильтрами 3x3 с дополнением нулями Р=1
4. ReLU
5. Полносвязный слой (со смещением) для вычисления оценок scores 10 классов

Вам не нужно делать какие-либо настройки гиперпараметров, но вы должны получить точность выше 43% после одной эпохи обучения.

In [17]:
def three_layer_convnet_init():
    """
    Инициализирует веса трехслойной ConvNet, для использования с
    three_layer_convnet, определенной выше.
    
     Входы: Отсутствуют
    
     Возвращает список, содержащий:
     - conv_w1: TensorFlow переменная, содержащая веса для первого слоя conv
     - conv_b1: переменная TensorFlow, содержащая смещения для первого слоя conv
     - conv_w2: TensorFlow Переменная, содержащая веса для второго слоя conv
     - conv_b2: переменная TensorFlow, содержащая смещения для второго слоя conv
     - fc_w: TensorFlow Переменная, содержащая веса  для полносвязанного слоя
     - fc_b: переменная TensorFlow, содержащая смещения для полносвязанного слоя
    """
    params = None
    ############################################################################
    # ЗАДАНИЕ: Инициализаровать параметры 3-х слойной сети                     #
    ############################################################################
    conv_w1 = tf.Variable(kaiming_normal((5, 5, 3, 32)))
    conv_b1 = tf.Variable(tf.zeros((32,)))
    conv_w2 = tf.Variable(kaiming_normal((3, 3, 32, 16)))
    conv_b2 = tf.Variable(tf.zeros((16,)))
    fc_w = tf.Variable(kaiming_normal((32 * 32 * 16, 10)))
    fc_b = tf.Variable(tf.zeros((10,)))
    params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
    ############################################################################
    #                             КОНЕЦ ВАШЕГО КОДА                            #
    ############################################################################
    return params

learning_rate = 3e-3
train_part2(three_layer_convnet, three_layer_convnet_init, learning_rate)

Iteration 0, loss = 3.5616
Got 77 / 1000 correct (7.70%)
Iteration 100, loss = 1.9414
Got 352 / 1000 correct (35.20%)
Iteration 200, loss = 1.5771
Got 390 / 1000 correct (39.00%)
Iteration 300, loss = 1.7180
Got 387 / 1000 correct (38.70%)
Iteration 400, loss = 1.7665
Got 433 / 1000 correct (43.30%)
Iteration 500, loss = 1.7898
Got 439 / 1000 correct (43.90%)
Iteration 600, loss = 1.6822
Got 462 / 1000 correct (46.20%)
Iteration 700, loss = 1.6048
Got 453 / 1000 correct (45.30%)


# Часть III: Keras модель API
Реализация нейронной сети с использованием базового API TensorFlow - это хороший способ понять, как работает TensorFlow, но несколько не удобно - нам пришлось вручную отслеживать все тензоры, у которых есть обучаемые параметры, и нам пришлось использовать зависимости управления для реализации шага обновления алгоритма SGD. Это не сложно для небольшой сети, но усложняется при большой модели нейросети.

К счастью, TensorFlow предоставляет пакеты более высокого уровня, такие как `tf.keras` и` tf.layers`, которые упрощают создание моделей из модульных объектно-ориентированных слоев; `tf.train` позволяет легко обучать эти модели с помощью различных алгоритмов оптимизации.

В этой части блокнота мы определим модели нейронных сетей, используя API интерфейс высокого уровня `tf.keras.Model`. Чтобы реализовать свою собственную модель, вам необходимо сделать следующее:

1. Определите новый класс, который является подклассом `tf.keras.model`. Присвойте вашему классу соответсвующее имя, которое указывает его назначение, например «TwoLayerFC» или «ThreeLayerConvNet».
2. В инициализаторе `__init __ ()` нового класса определите все слои, которые вам нужны в качестве атрибутов класса. Пакет `tf.layers` предоставляет множество обобщенных нейросетевых слоёв, таких как` tf.layers.Dense` для полносвязанных слоев и `tf.layers.Conv2D` для сверточных слоев. Внутри эти слои будут создавать тензоры «Variable» для любых обучаемых параметров. **Предупреждение**: Не забудьте вызвать `super () .__ init __ ()` в качестве первой строки вашего инициализатора!
3. Реализуйте метод `call ()` для вашего класса; он осуществляет прямое распространение для вашей модели и определяет *связи*  вашей сети. Слои, определенные в `__init __ ()`, применяются в  `__call __ ()`, поэтому они могут использоваться как объекты функций, которые преобразуют входные тензоры в выходные тензоры. Не определяйте новые слои в `call ()`; любые слои, которые вы хотите использовать при прямом распространении, должны быть определены в `__init __ ()`.

После того, как вы определили свой подкласс `tf.keras.Model`, вы можете создать его экземпляр и использовать его подобно модели из части II.


### Модуль прикладного интерфейса (API): 2-х слойная сеть

Вот конкретный пример использования API `tf.keras.Model` для определения двухслойной сети. 

Мы используем объект `Initializer` для задания начальных значений обучаемых параметров слоев; в частности, `tf.variance_scaling_initializer` соответствует методу инициализации Kaiming (Хе), использованному в части II. Подробнее об этом можно узнать здесь: https://www.tensorflow.org/api_docs/python/tf/variance_scaling_initializer

Объект `tf.layers.Dense` используется для представления двух полносвязанных слоев модели. В дополнение к умножению входа на весовую матрицу и добавлению вектора смещения, этот слой также может обеспечить применение нелинейности. Для первого слоя ниже используется функция активации ReLU, для этого конструктору передается параметр `activation = tf.nn.relu`; во втором слое нелиненйость не применяется.

К сожалению, функция `flatten`, определенная в части II, несовместима с API` tf.keras.Model`; к счастью, мы можем использовать `tf.layers.flatten` для выполнения той же операции. Проблема с нашей функцией «уплощения» из части II связана со статическими или динамическими формами для тензоров, что выходит за рамки этого блокнота; вы можете больше узнать о различии [в документации] (https://www.tensorflow.org/programmers_guide/faq#tensor_shapes).

In [18]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super().__init__()        
        initializer = tf.variance_scaling_initializer(scale=2.0)
        self.fc1 = tf.layers.Dense(hidden_size, activation=tf.nn.relu,
                                   kernel_initializer=initializer)
        self.fc2 = tf.layers.Dense(num_classes,
                                   kernel_initializer=initializer)
    def call(self, x, training=None):
        x = tf.layers.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """Небольшой  тест для проверки модели TwoLayerFC выше"""
    tf.reset_default_graph()
    input_size, hidden_size, num_classes = 50, 42, 10

    # Как обычно, в TensorFlow сначала нужно определить вычислительный граф.
    # С этой целью мы сначала создадим объект TwoLayerFC, а затем используем 
    # его для получения scores.
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        x = tf.zeros((64, input_size))
        scores = model(x)

    # Теперь, когда наш вычислительный граф определен, мы можем выполнить вычисления на графе
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        scores_np = sess.run(scores)
        print(scores_np.shape)
        
test_TwoLayerFC()

(64, 10)


###  Функциональный  API: 2-х слойная нейронная сеть
Пакет `tf.layers` предоставляет два разных API  высокого уровня для определения моделей нейронных сетей. В приведенном выше примере мы использовали **объектно-ориентированный API**, где каждый слой нейронной сети представлен как объект Python (например, `tf.layers.Dense`). Здесь мы рассмотрим функциональный API, где каждый уровень представляет собой функцию Python (например, `tf.layers.dense`), входы и выходы которой являются тензорами TensorFlow и которая внутри использует тензоры в вычислительном графе для хранения любых обучаемых весов.

Чтобы построить сеть, нужно передать входной тензор на первый слой и последовательно построить последующие слои. Вот пример того, как построить ту же двухслойную сеть с функциональным API.


In [19]:
def two_layer_fc_functional(inputs, hidden_size, num_classes):     
    initializer = tf.variance_scaling_initializer(scale=2.0)
    flattened_inputs = tf.layers.flatten(inputs)
    fc1_output = tf.layers.dense(flattened_inputs, hidden_size, activation=tf.nn.relu,
                                 kernel_initializer=initializer)
    scores = tf.layers.dense(fc1_output, num_classes,
                             kernel_initializer=initializer)
    return scores

def test_two_layer_fc_functional():
    """Небольшой  тест для проверки модели TwoLayerFC выше"""
    tf.reset_default_graph()
    input_size, hidden_size, num_classes = 50, 42, 10

    
    # Как обычно, в TensorFlow сначала нужно определить вычислительный граф.
    # Для этого мы сначала построим двухслойный сетевой граф, вызвав
    # two_layer_network (). Эта функция строит вычислительный граф
    # и возвращает тензор рейтингов scores.
    with tf.device(device):
        x = tf.zeros((64, input_size))
        scores = two_layer_fc_functional(x, hidden_size, num_classes)

    # Теперь, когда наш вычислительный граф определен, мы можем выполнить вычисления на графе
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        scores_np = sess.run(scores)
        print(scores_np.shape)
        
test_two_layer_fc_functional()

(64, 10)


### Keras модель  API: 3-х слойная сеть  ConvNet
Теперь настало время реализовать трехслойную ConvNet с использованием API `tf.keras.Model`. Модель должна иметь ту же архитектуру, что и ранее в части II:

1. Сверточный слой с 5 х 5 фильтрами и  с дополнением нулями Р=2
2. Нелинейность ReLU
3. Сверточный слой с 3 x 3 фильтрами и  с дополнением нулями Р=1
4. Нелинейность ReLU
5. Полносвязный слой, формирующий рейтинги  классов scores

Вы должны инициализировать веса сети, используя тот же метод инициализации, который использовался в двухслойной сети выше.

**Совет**: обратитесь к документации для `tf.layers.Conv2D` и` tf.layers.Dense`:

https://www.tensorflow.org/api_docs/python/tf/layers/Conv2D

https://www.tensorflow.org/api_docs/python/tf/layers/Dense

In [20]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # ЗАДАНИЕ:                                                             #
        # Реализуйте метод __init__ для трехслойной ConvNet. Вы должys создать #
        # объекты слоя, которые будут использоваться при прямом распространении#
        ########################################################################
        initializer = tf.variance_scaling_initializer(scale=2.0)
        self.conv1 = tf.layers.Conv2D(channel_1,[5,5],[1,1],padding='same',activation=tf.nn.relu,
                                      kernel_initializer=initializer)
        self.conv2 = tf.layers.Conv2D(channel_2,[3,3],[1,1],padding='same',activation=tf.nn.relu,
                                      kernel_initializer=initializer)
        self.fc = tf.layers.Dense(num_classes,kernel_initializer=initializer)
        ########################################################################
        #                           КОНЕЦ ВАШЕГО КОДА                          #
        ########################################################################
        
    def call(self, x, training=None):
        scores = None
        ########################################################################
        # ЗАДАНИЕ: выполнить прямое распространение для 3-х слойной ConvNet.   #
        # Используйте объекты слоя, определенные в методе __init__.            #
        ########################################################################
        x = self.conv1(x)
        x = self.conv2(x)
        x = tf.layers.flatten(x)
        scores = self.fc(x)
        ########################################################################
        #                          КОНЕЦ ВАШЕГО КОДА                           #
        ########################################################################        
        return scores

После завершения реализации «ThreeLayerConvNet» выше вы можете запустить код ниже, чтобы убедиться, что ваша реализация не сбоит и возвращает выходы ожидаемой формы.

In [21]:
def test_ThreeLayerConvNet():
    tf.reset_default_graph()
    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        scores_np = sess.run(scores)
        print(scores_np.shape)

test_ThreeLayerConvNet()

(64, 10)


### Keras модель API: цикл обучения
Нам нужно реализовать несколько иной цикл обучения, когда используем API `tf.keras.Model`. Вместо того, чтобы вычислять градиенты и обновлять веса вручную, будем использовать объект `Optimizer` из пакета` tf.train`, который позаботится об всех деталях. Вы можете узнать больше об `Optimizer` здесь:
https://www.tensorflow.org/api_docs/python/tf/train/Optimizer

In [22]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1):
    """
    
    Простой цикл обучения для использования с моделями, определенными с помощью tf.keras. 
    Он обучает модель на одной эпохе на мн-ве CIFAR-10 и периодически проверяет
    точность на валидационном мн-ве CIFAR-10.
    
     Входы:
     - model_init_fn: функция, которая не принимает никаких параметров; когда её вызывают
       создает модель, которую мы хотим обучить: model = model_init_fn ()
     - optimizer_init_fn: функция, которая не принимает никаких параметров; когда его вызывают
       создает объект Optimizer, который мы будем использовать для оптимизации модели:
       optimizer = optimizer_init_fn ()
     - num_epochs: количество эпох обучения
    
     Возвращает: ничего не возвращает, но выводит ход обучения
      
    """
    tf.reset_default_graph()    
    with tf.device(device):
        # Cтроим вычислительный граф, который будем использовать для обучения модели. 
        # Используем model_init_fn для создания модели, объявляем плейсхолдеры для
        # данных и меток
        x = tf.placeholder(tf.float32, [None, 32, 32, 3])
        y = tf.placeholder(tf.int32, [None])
                
        # Нам нужен особый плейсхолдер, чтобы явно указать, находится ли модель в фазе обучения
        # или нет. Это связано с тем, что ряд слоев ведет себя по-разному в
        # ходе обучения и в ходе тестирования, например, dropout и блочная нормализация.
        # Мы передаем эту переменную в граф через feed_dict, как показано ниже.
        is_training = tf.placeholder(tf.bool, name='is_training')
                
        # Используем  модель, чтобы сделать прямое распространение
        scores = model_init_fn(x, is_training)

        # Вычисляем потери
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        loss = tf.reduce_mean(loss)
        
        # Используем optimizer_fn для создания объекта Optimizer, затем используем его
        # для настроики этапа обучения. Попросим TensorFlow оценить
        # train_op, возвращаемый optimizer.minimize(loss), что заставит сделать
        # один шаг обновления, используя текущий мини-блок данных.
        
        # Обратите внимание, что мы используем tf.control_dependencies, чтобы заставить модель
        # запускать tf.GraphKeys.UPDATE_OPS на каждом шаге обучения. tf.GraphKeys.UPDATE_OPS
        # содержит операторы, которые обновляют состояния сети.
        # Например, функция tf.layers.batch_normalization добавляет операторы 
        # обновления текущего среднего и дисперсии для tf.GraphKeys.UPDATE_OPS.
        
        optimizer = optimizer_init_fn()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)

    # Теперь мы можем много раз запускать вычислительный граф для обучения модели.
    # Когда мы вызываем sess.run, мы просим оценить train_op, что приводит
    # к обновлению парметров модели.
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        t = 0
        for epoch in range(num_epochs):
            print('Starting epoch %d' % epoch)
            for x_np, y_np in train_dset:
                feed_dict = {x: x_np, y: y_np, is_training:1}
                loss_np, _ = sess.run([loss, train_op], feed_dict=feed_dict)
                if t % print_every == 0:
                    print('Iteration %d, loss = %.4f' % (t, loss_np))
                    check_accuracy(sess, val_dset, x, scores, is_training=is_training)
                    print()
                t += 1

### Keras модель API: обучение 2-х слойной нейросети

Теперь мы можем использовать инструменты, определенные выше, для обучения двухслойной сети на мн-ве данных CIFAR-10. Определим `model_init_fn` и` optimizer_init_fn`, которые создают модель и оптимизатор при вызове. Будем обучать модель с помощью простого стохастического градиентного спуска. Для этого определим функцию `tf.train.GradientDescentOptimizer`; вы можете
[прочитать это здесь](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer).

Вам не требуется  настраивать гиперпараметры, но вы должны достичь точности выше 40% после одной эпохи обучения.


In [23]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn(inputs, is_training):
    return TwoLayerFC(hidden_size, num_classes)(inputs)

def optimizer_init_fn():
    return tf.train.GradientDescentOptimizer(learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Starting epoch 0
Iteration 0, loss = 2.9074
Got 119 / 1000 correct (11.90%)

Iteration 100, loss = 1.8721
Got 366 / 1000 correct (36.60%)

Iteration 200, loss = 1.4586
Got 389 / 1000 correct (38.90%)

Iteration 300, loss = 1.7800
Got 341 / 1000 correct (34.10%)

Iteration 400, loss = 1.8933
Got 419 / 1000 correct (41.90%)

Iteration 500, loss = 1.7896
Got 427 / 1000 correct (42.70%)

Iteration 600, loss = 1.8617
Got 432 / 1000 correct (43.20%)

Iteration 700, loss = 1.8906
Got 429 / 1000 correct (42.90%)



### Keras модель API: обучениие 2-х слойной нейросети (функциональный API)
Аналогичным образом обцчаем двухслойную сеть, построенную с использованием функционального API.

In [24]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn(inputs, is_training):
    return two_layer_fc_functional(inputs, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.train.GradientDescentOptimizer(learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Starting epoch 0
Iteration 0, loss = 3.0776
Got 100 / 1000 correct (10.00%)

Iteration 100, loss = 1.8249
Got 378 / 1000 correct (37.80%)

Iteration 200, loss = 1.5358
Got 404 / 1000 correct (40.40%)

Iteration 300, loss = 1.8241
Got 391 / 1000 correct (39.10%)

Iteration 400, loss = 1.8732
Got 416 / 1000 correct (41.60%)

Iteration 500, loss = 1.7844
Got 448 / 1000 correct (44.80%)

Iteration 600, loss = 1.8352
Got 441 / 1000 correct (44.10%)

Iteration 700, loss = 1.9356
Got 450 / 1000 correct (45.00%)



### Keras модель API: Обучение 3-х слойной сети ConvNet
Здесь Вы должны использовать инструменты, которые определили выше, для обучения трехслойной ConvNet на множестве данных CIFAR-10. ConvNet должна использовать 32 фильтра в первом сверточном слое и 16 фильтров во втором слое.

Для обучения модели вы должны использовать градиентный спуск с моментом Нестерова 0.9.

**СОВЕТ**: https://www.tensorflow.org/api_docs/python/tf/train/MomentumOptimizer

Вам не нужно выполнять выбор гиперпараметров, но Вы должны достичь точности выше 45% после обучения в течение одной эпохи.

In [29]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn(inputs, is_training):
    model = None
    ############################################################################
    # ЗАДАНИЕ: Завершите реализацию model_fn.                                  #
    ############################################################################
    model = ThreeLayerConvNet(channel_1,channel_2,num_classes)
    ############################################################################
    #                           КОНЕЦ ВАШЕГО КОДА                              #
    ############################################################################
    return model(inputs)

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # ЗАДАНИЕ: Завершите реализацию model_fn.                                  #
    ############################################################################
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True)
    ############################################################################
    #                           КОНЕЦ ВАШЕГО КОДА                              #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Starting epoch 0
Iteration 0, loss = 2.8935
Got 78 / 1000 correct (7.80%)

Iteration 100, loss = 1.7071
Got 401 / 1000 correct (40.10%)

Iteration 200, loss = 1.3405
Got 478 / 1000 correct (47.80%)

Iteration 300, loss = 1.4474
Got 486 / 1000 correct (48.60%)

Iteration 400, loss = 1.3130
Got 502 / 1000 correct (50.20%)

Iteration 500, loss = 1.4488
Got 543 / 1000 correct (54.30%)

Iteration 600, loss = 1.4090
Got 543 / 1000 correct (54.30%)

Iteration 700, loss = 1.3727
Got 537 / 1000 correct (53.70%)



# Часть IV:  последовательный API-интерфейс  Keras
В третьей части мы ввели API-интерфейс `tf.keras.Model`, который позволяет определять модели с любым количеством обучаемых слоев и с произвольной связностью между слоями.

Однако для многих моделей не нужна такая гибкость - многие модели могут быть выражены в виде последовательного стека слоев, причем выход одного слоя подается на следующего слоя в качестве входа. Если  модель соответствует этому шаблону, то существует простой способ определения модели с помощью `tf.keras.Sequential`. В этом случае не нужно писать какие-либо пользовательские классы; просто вызывается конструктор `tf.keras.Sequential` со списком, содержащим последовательность объектов слоя.

Одна из сложностей использовния `tf.keras.Sequential` заключается то, что нужно определить форму входного тензора для модели, передав значение `input_shape` первого слоя модели.

### Последовательный API-интерфейс Keras: 2-х слойная сеть
Перепишем двухслойную полносвязанную сеть с помощью `tf.keras.Sequential` и обучим ее с использованием цикла обучения, определенного выше.

Вам не нужно выполнять выбор гиперпараметров, но вы должны видеть точность выше 40% после обучения сети в течение одной эпохи.

In [27]:
learning_rate = 1e-2

def model_init_fn(inputs, is_training):
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.variance_scaling_initializer(scale=2.0)
    layers = [
        tf.layers.Flatten(input_shape=input_shape),
        tf.layers.Dense(hidden_layer_size, activation=tf.nn.relu,
                        kernel_initializer=initializer),
        tf.layers.Dense(num_classes, kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model(inputs)

def optimizer_init_fn():
    return tf.train.GradientDescentOptimizer(learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Starting epoch 0
Iteration 0, loss = 2.9806
Got 107 / 1000 correct (10.70%)

Iteration 100, loss = 2.0099
Got 360 / 1000 correct (36.00%)

Iteration 200, loss = 1.4026
Got 390 / 1000 correct (39.00%)

Iteration 300, loss = 1.7913
Got 379 / 1000 correct (37.90%)

Iteration 400, loss = 1.7431
Got 422 / 1000 correct (42.20%)

Iteration 500, loss = 1.7702
Got 418 / 1000 correct (41.80%)

Iteration 600, loss = 1.8627
Got 429 / 1000 correct (42.90%)

Iteration 700, loss = 2.0135
Got 440 / 1000 correct (44.00%)



### Последовательный API-интерфейс Keras: 3-х слойная сеть ConvNet
Здесь вы должны использовать `tf.keras.Sequential` для переопределения одной и той же трехуровневой архитектуры ConvNet, используемой в Части II и Части III. Напоминаем, что ваша модель должна иметь следующую архитектуру:

1. Сверточный слой с  16 фильтрами 5x5 с дополнением нулями Р=2
2. Нелинейность ReLU
3. Сверточный слой с 32 фильтрами 3x3 с дополнением нулями Р=1
4. Нелинейность ReLU
5. Полносвязный слой, оценивающий рейтинги классов scores

Необходимо инициализировать весовые коэффициенты модели с помощью `tf.variance_scaling_initializer`, как указано выше.

Модель необходимо обучить  с использованием момента Нестерова 0.9.

Вам не нужно выполнять выбор гиперпараметров, но вы должны достичь точности выше 45% после обучения в течение одной эпохи.

In [28]:
def model_init_fn(inputs, is_training):
    model = None
    ############################################################################
    # ЗАДАНИЕ: Создать 3-х слойную  ConvNet, испольуя tf.keras.Sequential.     #
    ############################################################################
    input_shape = (32, 32, 3)
    channel_1, channel_2, num_classes = 32, 16, 10
    initializer = tf.variance_scaling_initializer(scale=2.0)
    layers = [
         # 'Same' padding acts similar to zero padding of 2 for this input
         tf.layers.Conv2D(channel_1,[5,5],strides=1, \
                                 padding="same", activation=tf.nn.relu,\
                                 kernel_initializer = initializer,input_shape=(32, 32,3)),
         tf.layers.Conv2D(channel_2,[3,3],strides=1, \
                                 padding="same", activation=tf.nn.relu,\
                                 kernel_initializer = initializer),
         tf.layers.Flatten(input_shape=input_shape),
         tf.layers.Dense(num_classes, kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    ############################################################################
    #                            КОНЕЦ ВАШЕГО КОДА                             #
    ############################################################################
    return model(inputs)

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # ЗАДАНИЕ: Завершить реализацию model_fn.                                  #
    ############################################################################
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True)
    ############################################################################
    #                           КОНЕЦ ВАШЕГО КОДА                              #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Starting epoch 0
Iteration 0, loss = 2.8855
Got 88 / 1000 correct (8.80%)

Iteration 100, loss = 1.8661
Got 357 / 1000 correct (35.70%)

Iteration 200, loss = 1.5427
Got 411 / 1000 correct (41.10%)

Iteration 300, loss = 1.6745
Got 435 / 1000 correct (43.50%)

Iteration 400, loss = 1.5944
Got 464 / 1000 correct (46.40%)

Iteration 500, loss = 1.6358
Got 492 / 1000 correct (49.20%)

Iteration 600, loss = 1.6221
Got 484 / 1000 correct (48.40%)

Iteration 700, loss = 1.6127
Got 498 / 1000 correct (49.80%)



# Часть V: CIFAR-10 - открытые проблемы

В этом разделе вы можете поэкспериментировать с любой архитектурой ConvNet, которую вы хотели бы использовать на CIFAR-10.

Вы должны поэкспериментировать с архитектурами, гиперпараметрами, функциями потерь, регуляризацией или чем-либо еще, что вы можете придумать для обучения модели, которая достигает **не менее 70%** точности на **валидационном** множестве в течение 10 эпох. Вы можете использовать функции `check_accuracy` и` train` определенные выше, или вы можете реализовать свой собственный цикл обучения.

Опишите, что вы делали в конце этого блокнота.

### С чем Вам следует экспериментировать:
- **Размер фильтра**: Выше мы использовали 5x5 и 3x3; это оптимально?
- **Количество фильтров**: Выше мы использовали 16 и 32 фильтра. Будет ли большее или меньшее число лучше?
- **Пулинг**: Мы не использовали никакого пулинга выше. Будет ли пулинг улучшать модель?
- **Нормализация**: улучшится ли ваша модель при использовании блочной нормализации, нормализации на слое, нормализации группы или какой-либо иной стратегии нормализации?
- **Архитектура**: В приведенной выше ConvNet имеется только три уровня обучаемых параметров. Будет ли более глубокая модель работать лучше?
- **Глобальный усредняющий пулинг**: вместо "уплощения" данных после последнего сверточного слоя будет ли глобальный усредняющий пулинг лучше? Эта стратегия используется, например, в  сети Google Inception  и в Остаточных (Residual) сетях.
- **Регуляризация**: Будет ли какая-то регуляризация повышать эффективность сети? Может быть, затухание весов или dropout?


### ПРЕДУПРЕЖДЕНИЕ: Блочная нормализация/ Dropout
Блочная нормализация (Batch Normalization) и Dropout **БУДУТ РАБОТАТЬ НЕ ПРАВИЛЬНО**, если вы используете функцию `train_part34()` с объектно-ориентированными API-интерфейсами `tf.keras.Model` или `tf.keras.Sequential`; если вы хотите добавить слои нормализации в цикл обучения, то Вы **должны использовать функциональный API-интерфейс tf.layers**.

Мы написали `train_part34 ()`, чтобы явно продемонстрировать, как работает TensorFlow; однако есть некоторые тонкости, которые затрудняют обработку объектно-ориентированного слоя блочной нормализации  в простом цикле обучения. На практике оба модуля `tf.keras` и` tf` предоставляют API высокого уровня, которые создают цикл обучения для вас, например [keras.fit] (https://keras.io/models/sequential/) и [tf. Estimator] (https://www.tensorflow.org/programmers_guide/estimators), оба из них будут корректно осуществлять блочную нормализацию  при использовании объектно-ориентированного API.

### Подсказки для обучения
Для каждой сетевой архитектуры, с которой вы экспериментируюте, вы должны выбрать скорость обучения и другие гиперпараметры. При этом есть несколько важных моментов, которые нужно иметь в виду:

- Если параметры работают хорошо, вы должны видеть улучшение в течение нескольких сотен итераций;
- Помните о грубой и тонкой настройке гиперпараметров: начните с проверки  гиперпараметров в широком диапазоне   всего на нескольких обучающих итерациях, чтобы найти комбинации параметров, которые работают вообще;
- После того, как вы найдете несколько наборов параметров, которые работают,  найдите их более точные значения. Возможно, вам придется проводить обучение при большом числе эпох.
- Вы должны использовать валидационное множество для поиска гиперпараметров.

### Движемся все выше и выше
Если вы ощущаете энтузиазм, есть много других функций, которые можно реализовать, чтобы попытаться повысить эффективность. Вы **не обязаны** их реализовывать, но не упустите шанс, если у вас есть время!

- Альтернативные оптимизаторы: вы можете попробовать Адам, Адаград, RMSprop и т. д.;
- Альтернативные функции активации, такие как  ReLU с утечкой, параметрическое ReLU, ELU или MaxOut;
- Ансамбли моделей;
- Расширение набора данных;
- Новые архитектуры.

- [ResNets](https://arxiv.org/abs/1512.03385) , где вход  предыдущего слоя добавляется к выходу.
  - [DenseNets](https://arxiv.org/abs/1608.06993) , где входы предыдущих слоев объединяются вместе.
  - [Этот блог содержит подробный обзор](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)
  
### Успешного обучения! 

In [7]:
from data import get_data_set
from model import model, lr
from time import time

tf.reset_default_graph()

train_x, train_y = get_data_set("train")
test_x, test_y = get_data_set("test")
tf.set_random_seed(21)
x, y, output, y_pred_cls, global_step, learning_rate = model()
global_accuracy = 0
epoch_start = 0


# PARAMS
_BATCH_SIZE = 128
_EPOCH = 15
_SAVE_PATH = "./tensorboard/cifar-10-v1.0.0/"


# LOSS AND OPTIMIZER
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                   beta1=0.9,
                                   beta2=0.999,
                                   epsilon=1e-08).minimize(loss, global_step=global_step)


# PREDICTION AND ACCURACY CALCULATION
correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# SAVER
merged = tf.summary.merge_all()
saver = tf.train.Saver()
sess = tf.Session()
train_writer = tf.summary.FileWriter(_SAVE_PATH, sess.graph)


try:
    print("\nTrying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    saver.restore(sess, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except ValueError:
    print("\nFailed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())


def train(epoch):
    global epoch_start
    epoch_start = time()
    batch_size = int(math.ceil(len(train_x) / _BATCH_SIZE))
    i_global = 0

    for s in range(batch_size):
        batch_xs = train_x[s*_BATCH_SIZE: (s+1)*_BATCH_SIZE]
        batch_ys = train_y[s*_BATCH_SIZE: (s+1)*_BATCH_SIZE]

        start_time = time()
        i_global, _, batch_loss, batch_acc = sess.run(
            [global_step, optimizer, loss, accuracy],
            feed_dict={x: batch_xs, y: batch_ys, learning_rate: lr(epoch)})
        duration = time() - start_time

        if s % 10 == 0:
            percentage = int(round((s/batch_size)*100))

            bar_len = 29
            filled_len = int((bar_len*int(percentage))/100)
            bar = '=' * filled_len + '>' + '-' * (bar_len - filled_len)

            msg = "Global step: {:>5} - [{}] {:>3}% - acc: {:.4f} - loss: {:.4f} - {:.1f} sample/sec"
            print(msg.format(i_global, bar, percentage, batch_acc, batch_loss, _BATCH_SIZE / duration))

    test_and_save(i_global, epoch)


def test_and_save(_global_step, epoch):
    global global_accuracy
    global epoch_start

    i = 0
    predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
    while i < len(test_x):
        j = min(i + _BATCH_SIZE, len(test_x))
        batch_xs = test_x[i:j, :]
        batch_ys = test_y[i:j, :]
        predicted_class[i:j] = sess.run(
            y_pred_cls,
            feed_dict={x: batch_xs, y: batch_ys, learning_rate: lr(epoch)}
        )
        i = j

    correct = (np.argmax(test_y, axis=1) == predicted_class)
    acc = correct.mean()*100
    correct_numbers = correct.sum()

    hours, rem = divmod(time() - epoch_start, 3600)
    minutes, seconds = divmod(rem, 60)
    mes = "\nEpoch {} - accuracy: {:.2f}% ({}/{}) - time: {:0>2}:{:0>2}:{:05.2f}"
    print(mes.format((epoch+1), acc, correct_numbers, len(test_x), int(hours), int(minutes), seconds))

    if global_accuracy != 0 and global_accuracy < acc:

        summary = tf.Summary(value=[
            tf.Summary.Value(tag="Accuracy/test", simple_value=acc),
        ])
        train_writer.add_summary(summary, _global_step)

        saver.save(sess, save_path=_SAVE_PATH, global_step=_global_step)

        mes = "This epoch receive better accuracy: {:.2f} > {:.2f}. Saving session..."
        print(mes.format(acc, global_accuracy))
        global_accuracy = acc

    elif global_accuracy == 0:
        global_accuracy = acc

    print("###########################################################################################################")


def train_custom_nn():
    train_start = time()

    for i in range(_EPOCH):
        print("\nEpoch: {}/{}\n".format((i+1), _EPOCH))
        train(i)

    hours, rem = divmod(time() - train_start, 3600)
    minutes, seconds = divmod(rem, 60)
    mes = "Best accuracy pre session: {:.2f}, time: {:0>2}:{:0>2}:{:05.2f}"
    print(mes.format(global_accuracy, int(hours), int(minutes), seconds))


train_custom_nn()
sess.close()


Trying to restore last checkpoint ...

Failed to restore checkpoint. Initializing variables instead.

Epoch: 1/60

Global step:     1 - [>-----------------------------]   0% - acc: 0.0938 - loss: 2.3014 - 54.0 sample/sec
Global step:    11 - [>-----------------------------]   3% - acc: 0.0859 - loss: 2.3058 - 57.8 sample/sec
Global step:    21 - [=>----------------------------]   5% - acc: 0.2109 - loss: 2.2199 - 92.2 sample/sec
Global step:    31 - [==>---------------------------]   8% - acc: 0.2969 - loss: 2.1591 - 34.7 sample/sec
Global step:    41 - [==>---------------------------]  10% - acc: 0.2031 - loss: 2.2281 - 97.7 sample/sec
Global step:    51 - [===>--------------------------]  13% - acc: 0.3359 - loss: 2.1453 - 35.0 sample/sec
Global step:    61 - [====>-------------------------]  15% - acc: 0.3594 - loss: 2.1018 - 95.4 sample/sec
Global step:    71 - [=====>------------------------]  18% - acc: 0.2578 - loss: 2.2002 - 44.0 sample/sec
Global step:    81 - [=====>--------

Global step:   742 - [==========================>---]  90% - acc: 0.4688 - loss: 1.9928 - 79.7 sample/sec
Global step:   752 - [==========================>---]  92% - acc: 0.5703 - loss: 1.8853 - 86.7 sample/sec
Global step:   762 - [===========================>--]  95% - acc: 0.5547 - loss: 1.9256 - 45.9 sample/sec
Global step:   772 - [============================>-]  97% - acc: 0.5391 - loss: 1.9208 - 64.3 sample/sec
Global step:   782 - [=============================>] 100% - acc: 0.5375 - loss: 1.9334 - 144.5 sample/sec

Epoch 2 - accuracy: 54.34% (5434/10000) - time: 00:14:58.77
This epoch receive better accuracy: 54.34 > 44.52. Saving session...
###########################################################################################################

Epoch: 3/60

Global step:   783 - [>-----------------------------]   0% - acc: 0.5469 - loss: 1.8875 - 39.9 sample/sec
Global step:   793 - [>-----------------------------]   3% - acc: 0.5703 - loss: 1.9161 - 27.1 sample/sec
Globa

Global step:  1454 - [====================>---------]  72% - acc: 0.6172 - loss: 1.8371 - 80.7 sample/sec
Global step:  1464 - [=====================>--------]  74% - acc: 0.5781 - loss: 1.8694 - 75.9 sample/sec
Global step:  1474 - [======================>-------]  77% - acc: 0.6094 - loss: 1.8369 - 69.5 sample/sec
Global step:  1484 - [======================>-------]  79% - acc: 0.5859 - loss: 1.8496 - 44.1 sample/sec
Global step:  1494 - [=======================>------]  82% - acc: 0.5859 - loss: 1.8832 - 92.2 sample/sec
Global step:  1504 - [========================>-----]  84% - acc: 0.6875 - loss: 1.7656 - 81.1 sample/sec
Global step:  1514 - [=========================>----]  87% - acc: 0.6172 - loss: 1.8229 - 42.3 sample/sec
Global step:  1524 - [==========================>---]  90% - acc: 0.5938 - loss: 1.8492 - 64.5 sample/sec
Global step:  1534 - [==========================>---]  92% - acc: 0.7109 - loss: 1.7513 - 85.8 sample/sec
Global step:  1544 - [========================

Global step:  2166 - [===============>--------------]  54% - acc: 0.7422 - loss: 1.7296 - 63.8 sample/sec
Global step:  2176 - [================>-------------]  56% - acc: 0.7266 - loss: 1.7195 - 36.6 sample/sec
Global step:  2186 - [=================>------------]  59% - acc: 0.7188 - loss: 1.7345 - 65.5 sample/sec
Global step:  2196 - [=================>------------]  61% - acc: 0.6719 - loss: 1.7773 - 79.7 sample/sec
Global step:  2206 - [==================>-----------]  64% - acc: 0.7188 - loss: 1.7446 - 41.6 sample/sec
Global step:  2216 - [===================>----------]  66% - acc: 0.7031 - loss: 1.7570 - 25.4 sample/sec
Global step:  2226 - [====================>---------]  69% - acc: 0.6875 - loss: 1.7802 - 26.5 sample/sec
Global step:  2236 - [====================>---------]  72% - acc: 0.6719 - loss: 1.7956 - 69.6 sample/sec
Global step:  2246 - [=====================>--------]  74% - acc: 0.6719 - loss: 1.7915 - 26.3 sample/sec
Global step:  2256 - [======================>-

Global step:  2888 - [===========>------------------]  38% - acc: 0.7422 - loss: 1.7160 - 77.3 sample/sec
Global step:  2898 - [===========>------------------]  41% - acc: 0.7734 - loss: 1.6935 - 90.9 sample/sec
Global step:  2908 - [============>-----------------]  43% - acc: 0.7656 - loss: 1.7071 - 39.8 sample/sec
Global step:  2918 - [=============>----------------]  46% - acc: 0.8203 - loss: 1.6367 - 93.7 sample/sec
Global step:  2928 - [==============>---------------]  49% - acc: 0.7500 - loss: 1.7055 - 94.6 sample/sec
Global step:  2938 - [==============>---------------]  51% - acc: 0.7109 - loss: 1.7510 - 50.4 sample/sec
Global step:  2948 - [===============>--------------]  54% - acc: 0.7500 - loss: 1.6887 - 95.6 sample/sec
Global step:  2958 - [================>-------------]  56% - acc: 0.8203 - loss: 1.6412 - 31.7 sample/sec
Global step:  2968 - [=================>------------]  59% - acc: 0.8203 - loss: 1.6462 - 91.6 sample/sec
Global step:  2978 - [=================>------

Global step:  3610 - [======>-----------------------]  23% - acc: 0.8516 - loss: 1.6091 - 94.2 sample/sec
Global step:  3620 - [=======>----------------------]  26% - acc: 0.8359 - loss: 1.6297 - 97.5 sample/sec
Global step:  3630 - [========>---------------------]  28% - acc: 0.7578 - loss: 1.7042 - 72.2 sample/sec
Global step:  3640 - [========>---------------------]  31% - acc: 0.8125 - loss: 1.6578 - 97.3 sample/sec
Global step:  3650 - [=========>--------------------]  33% - acc: 0.7734 - loss: 1.6932 - 66.7 sample/sec
Global step:  3660 - [==========>-------------------]  36% - acc: 0.7500 - loss: 1.7097 - 97.5 sample/sec
Global step:  3670 - [===========>------------------]  38% - acc: 0.7656 - loss: 1.6977 - 91.0 sample/sec
Global step:  3680 - [===========>------------------]  41% - acc: 0.7578 - loss: 1.6996 - 61.3 sample/sec
Global step:  3690 - [============>-----------------]  43% - acc: 0.7734 - loss: 1.6831 - 55.0 sample/sec
Global step:  3700 - [=============>----------

Global step:  4322 - [=>----------------------------]   5% - acc: 0.8672 - loss: 1.5952 - 60.6 sample/sec
Global step:  4332 - [==>---------------------------]   8% - acc: 0.8047 - loss: 1.6542 - 74.6 sample/sec
Global step:  4342 - [==>---------------------------]  10% - acc: 0.8125 - loss: 1.6469 - 27.9 sample/sec
Global step:  4352 - [===>--------------------------]  13% - acc: 0.8047 - loss: 1.6579 - 64.3 sample/sec
Global step:  4362 - [====>-------------------------]  15% - acc: 0.8047 - loss: 1.6519 - 44.0 sample/sec
Global step:  4372 - [=====>------------------------]  18% - acc: 0.8281 - loss: 1.6467 - 35.2 sample/sec
Global step:  4382 - [=====>------------------------]  20% - acc: 0.8438 - loss: 1.6204 - 74.1 sample/sec
Global step:  4392 - [======>-----------------------]  23% - acc: 0.8984 - loss: 1.5679 - 34.4 sample/sec
Global step:  4402 - [=======>----------------------]  26% - acc: 0.8516 - loss: 1.6071 - 58.8 sample/sec
Global step:  4412 - [========>---------------

Global step:  5073 - [============================>-]  97% - acc: 0.8672 - loss: 1.5937 - 71.9 sample/sec
Global step:  5083 - [=============================>] 100% - acc: 0.8625 - loss: 1.5884 - 44.2 sample/sec

Epoch 13 - accuracy: 75.52% (7552/10000) - time: 00:19:11.51
This epoch receive better accuracy: 75.52 > 75.18. Saving session...
###########################################################################################################

Epoch: 14/60

Global step:  5084 - [>-----------------------------]   0% - acc: 0.9062 - loss: 1.5560 - 31.5 sample/sec
Global step:  5094 - [>-----------------------------]   3% - acc: 0.7812 - loss: 1.6698 - 35.0 sample/sec
Global step:  5104 - [=>----------------------------]   5% - acc: 0.8906 - loss: 1.5749 - 48.6 sample/sec
Global step:  5114 - [==>---------------------------]   8% - acc: 0.8516 - loss: 1.6153 - 76.5 sample/sec
Global step:  5124 - [==>---------------------------]  10% - acc: 0.8438 - loss: 1.6138 - 65.0 sample/sec
Glob

Global step:  5785 - [======================>-------]  79% - acc: 0.7891 - loss: 1.6693 - 74.0 sample/sec
Global step:  5795 - [=======================>------]  82% - acc: 0.8359 - loss: 1.6204 - 45.3 sample/sec
Global step:  5805 - [========================>-----]  84% - acc: 0.9141 - loss: 1.5468 - 47.5 sample/sec
Global step:  5815 - [=========================>----]  87% - acc: 0.8281 - loss: 1.6279 - 73.6 sample/sec
Global step:  5825 - [==========================>---]  90% - acc: 0.8594 - loss: 1.6046 - 75.2 sample/sec
Global step:  5835 - [==========================>---]  92% - acc: 0.8438 - loss: 1.6177 - 77.0 sample/sec
Global step:  5845 - [===========================>--]  95% - acc: 0.8281 - loss: 1.6311 - 61.3 sample/sec
Global step:  5855 - [============================>-]  97% - acc: 0.8828 - loss: 1.5829 - 72.4 sample/sec
Global step:  5865 - [=============================>] 100% - acc: 0.8375 - loss: 1.6022 - 42.9 sample/sec

Epoch 15 - accuracy: 75.31% (7531/10000) - ti

KeyboardInterrupt: 

In [10]:
tf.reset_default_graph()

test_x, test_y = get_data_set("test")
x, y, output, y_pred_cls, global_step, learning_rate = model()


_BATCH_SIZE = 128
_CLASS_SIZE = 10
_SAVE_PATH = "./tensorboard/cifar-10-v1.0.0/"



saver = tf.train.Saver()
sess = tf.Session()


try:
    print("\nTrying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    saver.restore(sess, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except ValueError:
    print("\nFailed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())


def test_nn():
    i = 0
    predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
    while i < len(test_x):
        j = min(i + _BATCH_SIZE, len(test_x))
        batch_xs = test_x[i:j, :]
        batch_ys = test_y[i:j, :]
        predicted_class[i:j] = sess.run(y_pred_cls, feed_dict={x: batch_xs, y: batch_ys})
        i = j

    correct = (np.argmax(test_y, axis=1) == predicted_class)
    acc = correct.mean() * 100
    correct_numbers = correct.sum()
    print()
    print("Accuracy on Test-Set: {0:.2f}% ({1} / {2})".format(acc, correct_numbers, len(test_x)))


test_nn()


sess.close()


Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from ./tensorboard/cifar-10-v1.0.0/-5474
Restored checkpoint from: ./tensorboard/cifar-10-v1.0.0/-5474

Accuracy on Test-Set: 75.77% (7577 / 10000)


## Опишите, что Вы делали 
В приведенной ниже ячейке вы должны  объяснить, что вы делали, какие-либо дополнительные особенности, которые вы использовали, и / или любые графики, которые вы получили в процессе обучения и тестирования сети.

ЗАДАНИЕ: Опишите, что Вы делали

Была построена модель сети

----
- conv
- conv
- pool
- drop
----
- conv
- pool
- conv
- pool
- drop
----
- fc
- drop
- softmax